In [7]:
from typing import Dict
from absl import app
from absl import flags
from absl import logging

import android_env
from dm_env import specs
import numpy as np
import os


In [13]:
# 먼저 load 함수를 통해 안드로이드 환경을 불러옵니다. 
# 에뮬레이터가 실행됩니다.
env = android_env.load(
      emulator_path='~/Android/Sdk/emulator/emulator',
      android_sdk_root='~/Android/Sdk',
      android_avd_home='~/.android/avd',
      avd_name='my_avd',
      adb_path='~/Android/Sdk/platform-tools/adb',
      task_path=f'{os.curdir}/tasks/2048/classic_2048.textproto',
      run_headless=False)

In [14]:
from android_env.wrappers.discrete_action_wrapper import DiscreteActionWrapper
from android_env.wrappers.image_rescale_wrapper import ImageRescaleWrapper
from android_env.wrappers.float_pixels_wrapper import FloatPixelsWrapper
from android_env.wrappers.tap_action_wrapper import TapActionWrapper

def make_env(env):
    # env = ImageRescaleWrapper(env, zoom_factors=(0.0625, 0.0745),  grayscale=True)
    # print('-'*128)
    # print(env.action_spec())
    # print()
    # print(env.observation_spec())  
    
    env = TapActionWrapper(env, touch_only=True)
    print('-'*128)
    print(env.action_spec())
    print()
    print(env.observation_spec())  
    env = DiscreteActionWrapper(env, (6, 9), redundant_actions=True) # action touch grid: 54 blocks
    print('-'*128)
    print(env.action_spec())
    print()
    print(env.observation_spec())  
    

    return env



In [15]:
# task의 observation과 action에 대한 정보를 보겠습니다.
env = make_env(env)
action_spec = env.action_spec() 
obs_spec = env.observation_spec()

--------------------------------------------------------------------------------------------------------------------------------
{'action_id': DiscreteArray(shape=(), dtype=int32, name=action_id, minimum=0, maximum=161, num_values=162)}

{'pixels': Array(shape=(1920, 1080, 3), dtype=dtype('uint8'), name='pixels'), 'timedelta': Array(shape=(), dtype=dtype('int64'), name='timedelta'), 'orientation': Array(shape=(4,), dtype=dtype('uint8'), name='orientation')}


AssertionError: 

In [6]:
# 환경을 초기화합니다. 에피소드 처음부터 시작하게 됩니다.
_ = env.reset()

In [10]:
print(len(_))
print(_[0])
print(_[1])
print(_[2])

print(_.reward)
print(_.discount)
print(_.observation['pixels'].shape)
print(_.observation['pixels'][0])
print(_.observation['pixels'][0].shape)
print(_.observation['orientation'])
print(_.observation['timedelta'])

4
StepType.FIRST
0.0
0.0
0.0
0.0
(120, 80, 1)
[[187]
 [227]
 [205]
 [230]
 [228]
 [201]
 [244]
 [252]
 [251]
 [251]
 [254]
 [223]
 [209]
 [224]
 [212]
 [206]
 [234]
 [253]
 [251]
 [251]
 [251]
 [252]
 [244]
 [203]
 [235]
 [203]
 [219]
 [210]
 [253]
 [251]
 [251]
 [251]
 [251]
 [253]
 [212]
 [207]
 [235]
 [211]
 [219]
 [202]
 [244]
 [252]
 [251]
 [251]
 [253]
 [236]
 [202]
 [227]
 [230]
 [206]
 [212]
 [223]
 [253]
 [251]
 [251]
 [252]
 [244]
 [188]
 [206]
 [219]
 [233]
 [204]
 [215]
 [218]
 [253]
 [251]
 [251]
 [253]
 [213]
 [196]
 [215]
 [214]
 [236]
 [203]
 [219]
 [211]
 [253]
 [251]
 [251]
 [251]]
(80, 1)
[1 0 0 0]
1652688341272175


In [11]:
def get_random_action() -> Dict[str, np.ndarray]:
    """Returns a random AndroidEnv action."""
    action = {}
    for k, v in action_spec.items():
        if isinstance(v, specs.DiscreteArray):
            action[k] = np.random.randint(low=0, high=v.num_values, dtype=v.dtype)
        else:
            action[k] = np.random.random(size=v.shape).astype(v.dtype)
    return action

In [20]:
total_episodes = 1
for e in range(total_episodes):
    _ = env.reset()
    total_rewards = 0
    # Training
    for step in range(100):
        action = get_random_action()
        print(action)
        timestep = env.step(action=action)
        print(timestep.observation['timedelta'])
        reward = timestep.reward
        total_rewards += 0
        


{'action_id': 91}
66681
{'action_id': 61}
66756
{'action_id': 100}
66728
{'action_id': 2}
71893
{'action_id': 19}
89617
{'action_id': 146}
87436
{'action_id': 46}
95325
{'action_id': 140}
75546
{'action_id': 145}
106549
{'action_id': 120}
80371
{'action_id': 41}
66615
{'action_id': 69}
68566
{'action_id': 84}
75088
{'action_id': 59}
76877
{'action_id': 7}
76616
{'action_id': 34}
85345
{'action_id': 37}
94482
{'action_id': 34}
93926
{'action_id': 32}
101299
{'action_id': 107}
86354
{'action_id': 54}
76197
{'action_id': 63}
83026
{'action_id': 16}
72217
{'action_id': 157}
66811
{'action_id': 30}
66718
{'action_id': 131}
71063
{'action_id': 94}
66787
{'action_id': 138}
77728
{'action_id': 53}
77001
{'action_id': 1}
79070
{'action_id': 79}
86428
{'action_id': 70}
94025
{'action_id': 41}
85330
{'action_id': 154}
82612
{'action_id': 40}
76790
{'action_id': 129}
91972
{'action_id': 131}
114627
{'action_id': 26}
102689
{'action_id': 128}
83497
{'action_id': 132}
66827
{'action_id': 26}
92770
{